In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install configparser

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pymysql
import csv
import boto3
import configparser
from collections import OrderedDict

In [6]:
parser = configparser.ConfigParser(strict=False)

In [7]:
parser.read('pipeline.ini')

['pipeline.ini']

In [8]:
print(parser.sections())

['aws_boto_credentials', 'mysql_config']


In [9]:
hostname = parser.get("mysql_config", "hostname")
port = parser.get("mysql_config", "port")
username = parser.get("mysql_config", "username")
dbname = parser.get("mysql_config", "database")
password = parser.get("mysql_config", "password")

In [10]:
conn = pymysql.connect(host=hostname,user=username,password=password,db=dbname,port=int(port))

In [11]:
if conn is None:
    print("Error connecting to the MySQL database")
else:
    print("MySQL connection established!")

MySQL connection established!


In [12]:
m_query = "SELECT * FROM Orders;"
local_filename = "order_extract.csv"

In [13]:
m_cursor = conn.cursor()
m_cursor.execute(m_query)
results = m_cursor.fetchall()

In [14]:
with open(local_filename, 'w') as fp:
    csv_w = csv.writer(fp, delimiter='|')
    csv_w.writerows(results)

In [15]:
fp.close()
m_cursor.close()
conn.close()

Checking the binlogs

In [16]:
pip install mysql-replication

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysql-replication: filename=mysql_replication-1.0.9-py3-none-any.whl size=79885 sha256=f2a7c1ef77b11a5029c9a055085257709f77b2bb7692d781db3edf602c775220
  Stored in directory: /Users/pulsaragunawardhana/Library/Caches/pip/wheels/3b/13/61/233cf9edc37ac7cc917448600be7c3865a437dc915afec91ee
Successfully built mysql-replication
Note: you may need to restart the kernel to use updated packages.


In [17]:
from pymysqlreplication import BinLogStreamReader
from pymysqlreplication import row_event
import configparser
import pymysqlreplication

In [19]:
# get the MySQL connection info
parser = configparser.ConfigParser()
parser.read("pipeline.ini")
hostname = parser.get("mysql_config", "hostname")
port = parser.get("mysql_config", "port")

In [20]:
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config", "password")
mysql_settings = {
    "host": hostname,
    "port": int(port),
    "user": username,
    "passwd": password
}

In [21]:
b_stream = BinLogStreamReader(connection_settings = mysql_settings,server_id=100,only_events=[row_event.DeleteRowsEvent,row_event.WriteRowsEvent,row_event.UpdateRowsEvent])

In [22]:
for event in b_stream:
    event.dump()
    b_stream.close()

                       Setting The Variable Value BINLOG_ROW_METADATA = FULL, BINLOG_ROW_IMAGE = FULL.
                       By Applying this, provide properly mapped column information on UPDATE,DELETE,INSERT.
                        


In [23]:
b_stream